In [1]:
!nvidia-smi

Wed Jan 22 19:53:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   37C    P8              6W /  165W |    1211MiB /  16380MiB |     16%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import torch
import transformers
import peft
import datasets
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

device(type='cuda', index=0)

In [3]:
dataset_path = "/app/datasets/oberon/docs/bb_ru"
#model_name = "Qwen/Qwen2.5-Coder-7B"
model_name = 'MTSAIR/Cotype-Nano'
PACKED=False

In [4]:
only_model_name = model_name.split("/")[-1]
dataset_name = dataset_path.split("/")[-1]
sft_model_path = f"/app/models/{dataset_name}_{only_model_name}"

In [5]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
peft_config = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "mlp.down_proj",
        "self_attn.k_proj",
        "self_attn.o_proj",
        "mlp.up_proj",
        "self_attn.v_proj",
        "mlp.gate_proj",
        "self_attn.q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=peft.TaskType.CAUSAL_LM
)


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [7]:
def load_texts():
    file_names = [] 
    for subdir, dirs, files in os.walk(dataset_path):
        if not "/Docu/" in subdir:
            continue
        for file in files:
            file_names.append(os.path.join(subdir, file))
    texts = []
    for f in file_names:
        with open(f, 'r', encoding='utf-8') as file:
            texts.append(file.read())
    return texts
texts = load_texts()

In [8]:
block_size = 256  # Максимальная длина последовательности
stride = 32      # Шаг перекрытия    
def tokenize_with_stride(texts):
    tokenized_data = tokenizer(
        texts,
        max_length=block_size,
        truncation=True,
        stride=stride,
        return_overflowing_tokens=True,
        padding="max_length",  # Дополняем до block_size
        return_tensors="pt",   # Возвращаем тензоры PyTorch
    )
    tokenized_data["labels"] = tokenized_data["input_ids"]
    return datasets.Dataset.from_dict(tokenized_data).train_test_split(test_size=0.2)
    
lm_datasets = tokenize_with_stride(texts)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 1962
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 491
    })
})

In [9]:

if False:# probaby need to remove as obsolete
    print("USING PACKED DATASET")
    def tokenize_function(examples):
        return tokenizer(examples['text'])
    
    tokenized_dataset = dataset.map(tokenize_function)
    
    def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
            # customize this part to your needs.
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result
    
    lm_datasets = tokenized_dataset.map(
        group_texts,
        batched=True,
        batch_size=1000,
        num_proc=4,
    )

    

In [10]:
print(tokenizer.decode(lm_datasets["train"][99]["input_ids"]))

 и поля ряда (колонок) разделены табуляцией (09X).

ReadeTuple читает строку текста базы данных, и присваивает значение строки, занятое одним полем базы данных, каждому соответствующему полю шаблона.

AppendInstance прикрепляет копию шаблонного текста в конец выходного файла и затем заменяет все символы-заполнители содержимым соответствующих полей базы данных. Эти подстановки делаются от конца присоедяемого текста к началу, так что из/в-индексы  не становятся недействительными при подстановке. Это и объясняет тот факт, что список полей выстраивается в обратном порядке, последнее поле - первым.   

Заметим, что каждая подстановка получает атрибуты текста от первого перемещенного символа, т.е., если символ-заполнитель "<Name>" в полужирном шрифте замещается строкой "Joe", результирующей заменой будет


In [11]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

In [12]:
model = peft.get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820


In [13]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(message)s')

In [20]:
timestr = time.strftime("%Y%m%d-%H%M%S")
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = transformers.TrainingArguments(
    output_dir=f"./results/{timestr}/",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    bf16=torch.cuda.get_device_capability(torch.cuda.current_device())[0] >= 8,  
    gradient_accumulation_steps=8,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=32,
    warmup_steps=100, 
    #eval_steps=32, 
    #max_steps=512,
    
    #load_best_model_at_end=True,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()
#trainer.evaluate()

Epoch,Training Loss,Validation Loss
0,21.156200,2.101048


2025-01-22 22:10:32,414 Resetting dropped connection: huggingface.co
2025-01-22 22:10:32,741 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-22 22:10:32,898 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0


In [125]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 5.12


In [147]:
final_path = sft_model_path + f"_{timestr}"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

2025-01-22 19:29:31,304 Resetting dropped connection: huggingface.co
2025-01-22 19:29:31,786 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-22 19:29:31,987 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0


('/app/models/bb_ru_Cotype-Nano/tokenizer_config.json',
 '/app/models/bb_ru_Cotype-Nano/special_tokens_map.json',
 '/app/models/bb_ru_Cotype-Nano/vocab.json',
 '/app/models/bb_ru_Cotype-Nano/merges.txt',
 '/app/models/bb_ru_Cotype-Nano/added_tokens.json',
 '/app/models/bb_ru_Cotype-Nano/tokenizer.json')

In [128]:
reference_model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)

2025-01-22 19:57:55,994 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-22 19:58:20,616 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/generation_config.json HTTP/11" 200 0


In [156]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

def prompt_to_chat(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

def infer(model, prompt, l=100, use_chat = True, temperature=0.4, top_p = 0.8):
    if use_chat:
        prompt = prompt_to_chat(prompt)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=l,
        temperature=temperature, 
        top_p=top_p,
        do_sample=True ,  
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
    return response


prompts =  [
    'Как в BlackBox можно вывести данные на консоль?', 
    'Какой тип данных в Component Pascal используется для хранения целых чисел?', 
    'МОДУЛЬ i21егэDemo2010C4ru;', 
    'Component Pascal is Oberon microsystems refinement of?', 
    'Log.String(', 
    'Типом целой константы является'
]  # feel free to add a few more that are not 100% assiciated with Python

In [131]:
def draw_diff(use_chat,l):
    print(f"USING CHAT TEMPLATE = {use_chat}")
        
    baseline = [infer(reference_model, p, l = l, use_chat=use_chat) for p in prompts]
    check = [infer(model, p, l = l, use_chat=use_chat) for p in prompts]
    rows = []
    for i, prompt in enumerate(prompts):
        # replace placeholders in the format() arguments
        rows.append(row_template.format(prompt, baseline[i], check[i]))
    display(HTML(table_template.format('\n'.join(rows))))

In [153]:
draw_diff(True, 39)

USING CHAT TEMPLATE = True


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,Вопрос о выводе данных на консоль в контексте программирования с использованием фреймворка BlackBox может быть связан с различными языковами программ,"Чтобы напечатать что-то на стандартный поток вывода (консоль), используйте процедуры WriteChar, WriteInt или другие аналогичные."
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,"В Pascal, как и во многих других языках программирования, для хранения целых чисел обычно используется тип данных `integer`. Этот тип поддерживает различные операции ари","В языке программирования Component Pascal целые числа могут быть представлены как SHORTCHAR, CHAR или INTEGER. Символ 0X может использоваться для обозначения от"
`МОДУЛЬ i21егэDemo2010C4ru;`,"Ваш вопрос содержит строку, которая выглядит как часть имени модуля или класса в программировании. Однако, если это была попытка задать конкретный вопрос","Этот модуль предназначен для изучения примерами основных средств программирования в Блэкбоксе, а именно:  процедуры-фабри"
`Component Pascal is Oberon microsystems refinement of?`,"It seems you are asking about the relationship between Component Pascal and Oberon microsystems. To clarify, ""Component Pascal"" could refer to a specific product or software component developed by Oberon Microsystems","Oberon microsystems developed the language Component Pascal as a refinement and extension of the existing language Oberon. It was also refined in several ways to make it more user-friendly, safe, and"
`Log.String(`,"It seems like you're asking about the `String` method in a logging context, but your question is incomplete. The `String` method does not exist for strings in most programming languages; instead",ersion: 0.8 поставлена под модуль i21sysIn в целях разработки языка Оберон-3 (см.
`Типом целой константы является`,"Вопрос о ""типом целой константы"" может быть немного неоднозначным, так как термин ""константа"" в контексте программирования обычно","типа значения. Константа означает значение, которое не может быть изменено в процессе выполнения программы. Дополнительно модуль i21sysIn содержит процеду"


In [150]:
draw_diff(False, 39)

USING CHAT TEMPLATE = False


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,I'm trying to debug a Java application using the Eclipse IDE and want to see the output of the `System.out.println` statements in my code. How can I achieve this? In the documentation,"В частности, как это сделать для массива целых? Во-первых, нужно открыть поток вывода (ввода) в рабочий. Это делается"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,"В компоненте Pascal, для хранения целых чисел обычно используется тип `Int`. Этот тип представляет собой 32-битное беззнаковое целое число. Если","Для хранения целых чисел, таких как 123 или -456789, используются типы SHORTCHAR (длина 1 бай"
`МОДУЛЬ i21егэDemo2010C4ru;`,"// Простой модуль для демонстрации // Модуль должен быть подключен к системе #include ""i21egedemo2010c4ru","(* info21, 2010-10-07 *) IMPORT Log := StdLog, In := i21sysIn, Texts := AD"
`Component Pascal is Oberon microsystems refinement of?`,"Oberon is a high-level, multi-paradigm object-oriented programming language that was developed in the 1970s. It is known for its support for concurrency and distributed","or rather generalization of Component Information Systems (CIS) as proposed by the Object-Oriented Database Society. CIS provides a type and its metamodel, i.e., an abstract description of"
`Log.String(`,1024) 这个代码片段是用Go语言编写的，它试图将一个字符串截断为不超过1024字符的长度。在这个字符串被截断,"""Enter"" ); Log.Ln; END Do; END ObxCount0. Листинг 5-2. Пример, показывающий работу команды Count"
`Типом целой константы является`,"число π (пи), которое приближается к значению 3.14159... и не может быть точно выражено в виде дроби, так как это число","ее значение. Это типы INTEGER, LONGINT, SHORTCHAR, CHAR, BYTE, SHORTINT, INTEGER, LONG, SHORTREAL и REAL. CONST false, true Эти кон"


In [160]:
print(infer(model, 'Ты помошник по среде Component Pascal. Ответь на вопрос: Как мне вывести строку hello в окно лога?', 256, use_chat = True))

Чтобы выполнить команду, нужно сначала открыть окошко лога (двойной клик по имени модуля Log; или Файлы -> Открыть документ... / File->Open Dialog, затем нажать Ctrl+O). После этого можно напечатать что-нибудь в обычном текстовом редакторе и нажать Ctrl+0.

Если хотите, чтобы команда выполнялась после компиляции программы, то следует добавить её к меню Правка (Edit) через Инфо->Меню (Info->Menu Listers), либо напрямую открывать меню после компиляции:
		"Compile &List Modules"	""	"DevCompiler.ListModules"	"TextCmds.SelectionGuard"
После компиляции программ:
	"Compile &All"	""	"DevCompiler.CompileThis"	""
	"Compile &Module List"	""	"DevCompiler.CompileModuleList"	"TextCmds.SelectionGuard"
	"Com&pile Module List &As Assistant"	""	"DevCompiler.InitAndCompileModuleList"	"Text


In [155]:
checkpoint = transformers.AutoModelForCausalLM.from_pretrained("./results/checkpoint-4500")

2025-01-22 19:40:08,590 Resetting dropped connection: huggingface.co
2025-01-22 19:40:09,095 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-22 19:40:29,916 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/generation_config.json HTTP/11" 200 0


In [157]:
print(infer(checkpoint, 'Ты помошник по среде Component Pascal. Ответь на вопрос: Как мне вывести строку hello в окно лога?', 256, use_chat = True, temperature = 0.1, top_p = 0.3))

Для начала вам нужно открыть окно логи, если оно еще не открыто. Для этого нажмите Ctrl+O (или выберите File->Log). Затем выполните следующие шаги:

1. Нажмите Ctrl+N для создания нового текстового документа.
2. Вставьте в него строку "hello".
3. Выделите эту строку и кликните правой кнопкой мыши по выделенной строке. В появившемся контекстном меню выберите "Выполнить как" -> "Вывести в лог". 
4. Если возникает ошибка, проверьте правильность написания слова "hello".

Если все сделано правильно, то в окне лога появится сообщение об успешной печать строки hello.

Надеюсь, что эта инструкция была понятна. Если у вас есть дополнительные вопросы или предложения, пожалуйста, сообщите об этом.<|im_end|>
